# NN and CNN Implementation

In [1]:
pip install python-mnist

Note: you may need to restart the kernel to use updated packages.


## Load MNIST Data

In [2]:
# MNISTの読み込み
import numpy as np
from mnist import MNIST
mndata = MNIST("./le4nn/")
X_train, Y_train = mndata.load_training()
X_test, Y_test = mndata.load_testing()
X_train = np.array(X_train)
X_train = X_train.reshape((X_train.shape[0],1,28,28))
Y_train = np.array(Y_train)
X_test = np.array(X_test)
X_test = X_test.reshape((X_test.shape[0],1,28,28))
Y_test = np.array(Y_test)

## Normalize MNIST Data

In [3]:
# MNIST教師データの正規化
def normalize_mnist(x):
    x = x.astype(np.float32)
    x /= 255.0
    return x

X_train = normalize_mnist(X_train)
X_test = normalize_mnist(X_test)

## General Parameter

In [4]:
# シード値固定
np.random.seed(404)

# バッチサイズ
b = 100

# 入力層のノードの数
d = 28 * 28

# 中間層のノード数
m = 50

# 出力層のノードの数
c = 10

# 学習率
eta = 0.01

# エポック数
epoch = 30

# 教師データ数
n = 60000

# one-hot vector
T_train = np.eye(c)[Y_train]
T_test = np.eye(c)[Y_test]

## Basic Layer

In [5]:
# シグモイド関数
# in : x / out : y
class Sigmoid:
    def __init__(self):
        self.y = None
    
    def forward(self, x, train_flag=True):
        y = 1.0 / (1.0 + np.exp(-x))
        self.y = y
        return y
    
    def backward(self, dy):
        dx = dy * (1.0 - self.y) * self.y
        return dx

    
# ソフトマックス関数 & クロスエントロピー誤差
# in : x / out : y
class SoftmaxWithCrossEntropy:
    def __init__(self):
        self.y = None
        self.t = None
        self.loss = None
    
    def softmax(self, x):
        alpha = np.max(x, axis=1, keepdims=True)
        exp_x = np.exp(x - alpha)
        sum_exp_x = np.sum(exp_x, axis=1, keepdims=True)
        return exp_x / sum_exp_x
    
    def cross_entropy(self, y, t):
        # log(0)対策
        delta = 1e-7
        t_log_y = t * np.log(y + delta)
        return -np.sum(t_log_y) / y.shape[0]
    
    def forward(self, x, t, train_flag=True):
        self.y = self.softmax(x)
        self.t = t
        self.loss = self.cross_entropy(self.y, self.t)
        return self.y, self.loss
    
    def backward(self, dy=1):
        dx = (self.y - self.t) / self.t.shape[0]
        return dx

    
# 全結合層
# in : x, W, b / out : y
class Affine:
    def __init__(self, W, b):
        self.x = None
        self.W = W
        self.b = b
        self.dW = None
        self.db = None
        self.x_shape_origin = None
    
    def forward(self, x, train_flag=True):
        # CNNの時はxの形状が2次元じゃないのでその対策をしないといけない
        self.x_shape_origin = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x
        y = np.dot(self.x, self.W) + self.b
        # 今回はAffineの後にConvolutionとかが来ないので形を直さなくていい
        return y
    
    def backward(self, dy):
        dx = np.dot(dy, self.W.T)
        self.dW = np.dot(self.x.T, dy)
        self.db = np.sum(dy, axis=0)
        # 入力を元の形に戻す
        dx = dx.reshape(*self.x_shape_origin)
        return dx

    
# ReLU関数
# in : x / out : y
class ReLU:
    def __init__(self):
        self.mask = None
    
    def forward(self, x, train_flag=True):
        self.mask = (x <= 0)
        y = x.copy()
        y[self.mask] = 0
        return y
    
    def backward(self, dy):
        dy[self.mask] = 0
        dx = dy
        return dx

    
# Dropout
class Dropout:
    def __init__(self, rho=0.5):
        self.rho = rho
        self.mask = None

    def forward(self, x, train_flag=True):
        if train_flag:
            # 入力と同じshapeの行列に範囲[0.0,1.0)の一様分布で値を埋めて、ρより大きい要素だけTrue
            self.mask = np.random.rand(*x.shape) > self.rho
            y = x * self.mask
            return y
        else:
            return x * (1.0 - self.rho)

    def backward(self, dy):
        dx = dy * self.mask
        return dx
    
    
# Batch Normalization
class BatchNormalization:
    def __init__(self, gamma=1.0, beta=0.0):
        self.gamma = gamma
        self.beta = beta
        self.dgamma = None
        self.dbeta = None
        self.epsilon = 1e-7   
        self.xu = None
        self.std = None
        self.xhat = None
        # テスト時に使う平均と分散
        self.test_mean = None
        self.test_var = None
        # 普通のNNとCNNでは入力の形が異なるので処理が微妙に変わる
        self.input_shape = None
     
    def forward(self, x, train_flag=True):
        # 入力の形は覚えておく
        self.input_shape = x.shape
        # CNNならば形を2次元に合わせる
        if x.ndim != 2:
            n, ch, h, w = x.shape
            x = x.reshape(n, -1)
        # 学習時とテスト時で処理が異なる
        if train_flag:
            mean = np.mean(x, axis=0)
            xu = x - mean
            self.xu = xu
            self.mean = mean
            var = np.mean(xu**2, axis=0)
            std = np.sqrt(var + self.epsilon)
            self.std = std
            xhat = (x - mean) / std
            self.xhat = xhat
            y = self.gamma * xhat + self.beta
            # 最初の1回はテスト時に使う平均と分散を初期化
            if self.test_mean is None:
                self.test_mean = np.zeros(x.shape[1])
                self.test_var = np.zeros(x.shape[1])
            # テスト時に使う平均と分散を修正
            self.test_mean = 0.9 * self.test_mean + 0.1 * mean
            self.test_var = 0.9 * self.test_var + 0.1 * var
        else:
            std = np.sqrt(self.test_var + self.epsilon)
            a = self.gamma / std
            b = self.beta - self.gamma * self.test_mean / std
            y = a * x + b
        # CNNの場合は形を戻さないといけないので
        y = y.reshape(*self.input_shape)
        return y
    
    def backward(self, dy):
        # CNNならば形を2次元に合わせる
        if dy.ndim != 2:
            n, ch, h, w = dout.shape
            dy = dy.reshape(n, -1)
        # バッチサイズ
        b = dy.shape[1]
        dxhat = dy * self.gamma
        dvar = np.sum(dxhat * self.xu * (-0.5) / self.std**3, axis=0)
        dmean = (-np.sum(dxhat / self.std, axis=0) + dvar * np.sum((-2.0) * self.xu, axis=0) / b)
        dx = dxhat / self.std + dvar * 2.0 * self.xu / b + dmean / b
        dgamma = np.sum(dy * self.xhat, axis=0)
        self.dgamma = dgamma
        dbeta = np.sum(dy, axis=0)
        self.dbeta = dbeta
        # CNNの場合は形を戻さないといけないので
        dx = dx.reshape(*self.input_shape)
        return dx

## Optimizer

In [6]:
class SGD:
    def __init__(self, eta = 0.01):
        self.eta = eta
    
    def update(self, params, grads):
        # パラメータ更新
        for key in params.keys():
            params[key] -= self.eta * grads[key]

            
class MomentumSGD:
    def __init__(self, eta=0.01, alpha=0.9):
        self.eta = eta
        self.alpha = alpha
        self.dW = None
    
    def update(self, params, grads):
        if self.dW is None:
            self.dW = {}
            for key, val in params.items():
                self.dW[key] = np.zeros_like(val)
        # パラメータ更新
        for key in params.keys():
            self.dW[key] = self.alpha * self.dW[key] - self.eta * grads[key]
            params[key] += self.dW[key]

            
class AdaGrad:
    def __init__(self, eta=0.001, h0=1e-8):
        self.eta = eta
        self.h = None
        self.h0 = h0
    
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.full_like(val, self.h0)
        # パラメータ更新
        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.eta * grads[key] / np.sqrt(self.h[key])

            
class RMSProp:
    def __init__(self, eta=0.001, rho=0.9, epsilon=1e-8):
        self.eta = eta
        self.rho = rho
        self.epsilon = epsilon
        self.h = None
    
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
        # パラメータ更新
        for key in params.keys():
            self.h[key] = self.rho * self.h[key] + (1.0 - self.rho) * grads[key] * grads[key]
            params[key] -= self.eta * grads[key] / (np.sqrt(self.h[key]) + self.epsilon)

            
class AdaDelta:
    def __init__(self, rho=0.95, epsilon=1e-6):
        self.rho = rho
        self.epsilon = epsilon
        self.h = None
        self.s = None
    
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            self.s = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
                self.s[key] = np.zeros_like(val)
        # パラメータ更新
        for key in params.keys():
            self.h[key] = self.rho * self.h[key] + (1.0 - self.rho) * grads[key] * grads[key]
            dW = -(np.sqrt(self.s[key] + self.epsilon) * grads[key] / np.sqrt(self.h[key] + self.epsilon))
            self.s[key] = self.rho * self.s[key] + (1.0 - self.rho) * dW * dW
            params[key] += dW

            
class Adam:
    def __init__(self, alpha=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.alpha = alpha
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.t = 0
        self.m = None
        self.v = None
    
    def update(self, params, grads):
        if self.m is None:
            self.m = {}
            self.v = {}
            for key, val in params.items():
                self.m[key] = np.zeros_like(val)
                self.v[key] = np.zeros_like(val)
        # パラメータ更新
        for key in params.keys():
            self.t += 1
            self.m[key] = self.beta1 * self.m[key] + (1.0 - self.beta1) * grads[key]
            self.v[key] = self.beta2 * self.v[key] + (1.0 - self.beta2) * grads[key] * grads[key]
            mhat = self.m[key] / (1.0 - np.power(self.beta1, self.t))
            vhat = self.v[key] / (1.0 - np.power(self.beta2, self.t))
            params[key] -= self.alpha * mhat / (np.sqrt(vhat) + self.epsilon)

## Weight Initialization

In [7]:
# パラメータの初期値

# 活性化関数がSigmoidの時
def xavier(prev_size, param_shape):
    param = np.random.randn(*param_shape) * np.sqrt(1.0 / prev_size)
    return param

# 活性化関数がReLUの時
def he(prev_size, param_shape):
    param = np.random.randn(*param_shape) * np.sqrt(2.0 / prev_size)
    return param

## Layer for CNN

In [8]:
# 計算量を減らすために入力データとフィルターを行列に展開する関数
def im2col(input_data, filter_h, filter_w, stride=1, padding=0):
    input_size, input_ch, input_h, input_w = input_data.shape
    # 出力のサイズを計算
    output_h = (input_h + 2 * padding - filter_h) // stride + 1
    output_w = (input_w + 2 * padding - filter_w) // stride + 1
    # 入力にパディング処理をする
    img = np.pad(input_data, [(0,0), (0,0), (padding, padding), (padding, padding)], 'constant')
    # 求めたい行列を0初期化
    col = np.zeros((input_size, input_ch, filter_h, filter_w, output_h, output_w))  
    # imgの展開処理
    for y in range(filter_h):
        y_max = y + stride * output_h
        for x in range(filter_w):
            x_max = x + stride * output_w
            # y:y_max:stride : y～y_max-1の要素をstride個毎に取り出す
            # x:x_max:stride : x～x_max-1の要素をstride個毎に取り出す
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]
    """
    # 上の処理は実質これ（https://qiita.com/kurumen-b/items/236c6255959a266cefaa）
    # これは愚直なので4重ループとなり遅い
    for move_y in range(output_h):
        for move_x in range(output_w):
            for y in range(filter_h):
                for x in range(filter_w):
                    # move_yは高々output_h-1の値しかとらない（output_hで抑えられる）
                    # つまりy + stride * move_yは上記のy:y_max:strideと等価である
                    # y + stride * move_yのループはyからstride * output_h未満の範囲をstride個ごとに取るということなので
                    col[:, :, y, x, move_y, move_x] = img[:, :, y + stride * move_y, x + stride * move_x]
    """
    # colを入力データ数 * 出力の高さ * 出力の幅 * 入力のチャネル数 * フィルターの高さ * フィルターの幅に並べ替える
    # reshapeして縦 : 入力データ数 * 出力の高さ * 出力の幅, 横 : 入力のチャネル数 * フィルターの高さ * フィルターの幅の行列に変換
    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(input_size * output_h * output_w, -1)
    return col


# im2colの逆伝播版
def col2im(col, input_shape, filter_h, filter_w, stride=1, padding=0):
    input_size, input_ch, input_h, input_w = input_shape
    # 出力のサイズを計算
    output_h = (input_h + 2 * padding - filter_h) // stride + 1
    output_w = (input_w + 2 * padding - filter_w) // stride + 1
    # colは元々(入力データ数 * 出力の高さ * 出力の幅) * (入力のチャネル数 * フィルターの高さ * フィルターの幅)の行列(2次元)
    # colを入力データ数 * 出力の高さ * 出力の幅 * 入力のチャネル数 * フィルターの高さ * フィルターの幅にreshape(6次元)
    # さらに入力データ数 * 入力のチャネル数 * フィルターの高さ * フィルターの幅 * 出力の高さ * 出力の幅に並び替える
    col = col.reshape(input_size, output_h, output_w, input_ch, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)
    # 求めたい入力を0初期化
    img = np.zeros((input_size, input_ch, input_h + 2 * padding + stride - 1, input_w + 2 * padding + stride - 1))
    for y in range(filter_h):
        y_max = y + stride * output_h
        for x in range(filter_w):
            x_max = x + stride * output_w
            # im2colの時と逆にしただけ
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]
    # パディングを取っ払う
    input_data = img[:, :, padding:input_h + padding, padding:input_w + padding]
    return input_data


# 畳み込み層
class Convolution:
    def __init__(self, W, b, stride=1, padding=0):
        self.x = None
        self.W = W
        self.b = b
        self.dW = None
        self.db = None
        self.stride = stride
        self.padding = padding
        self.col = None
        self.col_W = None

    def forward(self, x, train_flag=True):
        self.x = x
        filter_size, channel, filter_h, filter_w = self.W.shape
        input_size, channel, input_h, input_w = x.shape
        # 出力のサイズを計算
        output_h = int((input_h + 2 * self.padding - filter_h) / self.stride) + 1
        output_w = int((input_w + 2 * self.padding - filter_w) / self.stride) + 1
        # 入力データを行列に展開（縦が入力データ数 * 出力の高さ * 出力の幅）
        col = im2col(x, filter_h, filter_w, self.stride, self.padding)
        self.col = col
        # フィルターを行列に展開（横がフィルター数）
        col_W = self.W.reshape(filter_size, -1).T
        self.col_W = col_W
        # 畳み込みを計算
        y = np.dot(col, col_W) + self.b
        # 入力データ数 * フィルター数 * 出力の高さ * 出力の幅の形に整える
        y = y.reshape(input_size, output_h, output_w, filter_size).transpose(0, 3, 1, 2)
        return y

    def backward(self, dy):
        filter_size, channel, filter_h, filter_w = self.W.shape
        # dyは元々入力データ数 * フィルター数 * 出力の高さ * 出力の幅(4次元)
        # dyを入力データ数 * 出力の高さ * 出力の幅 * フィルター数に並び替える
        # さらに(入力データ数 * 出力の高さ * 出力の幅) * フィルター数にreshape(2次元)
        dy = dy.transpose(0,2,3,1).reshape(-1, filter_size)
        # Affineのbackwardとほぼ同じ
        # dxにcol2imが、dWにreshapeが必要なところが違う
        dcol = np.dot(dy, self.col_W.T)
        dx = col2im(dcol, self.x.shape, filter_h, filter_w, self.stride, self.padding)
        self.dW = np.dot(self.col.T, dy)
        self.dW = self.dW.transpose(1, 0).reshape(filter_size, channel, filter_h, filter_w)
        self.db = np.sum(dy, axis=0)
        return dx


# プーリング層（maxプーリング）
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, padding=0):
        self.x = None
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.padding = padding
        self.max_id = None

    def forward(self, x, train_flag=True):
        self.x = x
        input_size, channel, input_h, input_w = x.shape
        # 出力のサイズを計算
        output_h = int(1 + (input_h - self.pool_h) / self.stride)
        output_w = int(1 + (input_w - self.pool_w) / self.stride)
        # 入力データを行列に展開
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.padding)
        # 縦が入力データ数 * 出力の高さ * 出力の幅 * チャネル数になるよう整形
        col = col.reshape(-1, self.pool_h * self.pool_w)
        # 行毎の最大値を持つ要素番号を求める（backwardで使うので）
        max_id = np.argmax(col, axis=1)
        self.max_id = max_id
        # 行毎の最大値を求める
        y = np.max(col, axis=1)
        # 入力データ数 * チャネル数 * 出力の高さ * 出力の幅の形に整える
        y = y.reshape(input_size, output_h, output_w, channel).transpose(0, 3, 1, 2)
        return y

    def backward(self, dy):
        pool_size = self.pool_h * self.pool_w
        # dyを入力データ数 * 出力の高さ * 出力の幅 * チャネル数に並び替える
        dy = dy.transpose(0, 2, 3, 1)
        # 最大値を与えた要素のインデックスについてのみ誤差を伝播したい
        # それ以外は0なのでまずは零行列を作る
        dz = np.zeros((dy.size, pool_size))
        # 最大値の要素を持っていたインデックスに対応する場所に誤差の値（dy）を埋め込む
        dz[np.arange(self.max_id.size), self.max_id.flatten()] = dy.flatten()
        # 形を整える
        dz = dz.reshape(dy.shape + (pool_size,)) 
        dcol = dz.reshape(dz.shape[0] * dz.shape[1] * dz.shape[2], -1)
        # 画像の形に戻す
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.padding)     
        return dx

## Sample NN 1

In [9]:
from collections import OrderedDict

class NormalNN:
    def __init__(self, d, m, c):
        # パラメータ初期化
        self.params = {}
        self.params['W1'] = xavier(d, (d, m))
        self.params['W2'] = xavier(m, (m, c))
        self.params['b1'] = xavier(d, (1, m))
        self.params['b2'] = xavier(m, (1, c))
        # NN生成
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Sigmoid'] = Sigmoid()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithCrossEntropy()
        
    # 順伝播
    def forward(self, x, t, tflag=True):
        for layer in self.layers.values():
            x = layer.forward(x, train_flag=tflag)
        y, loss = self.lastLayer.forward(x, t)
        return y, loss
    
    # 逆伝播
    def backpropagation(self):
        dy = 1.0
        dy = self.lastLayer.backward(dy)
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dy = layer.backward(dy)
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['W2'] = self.layers['Affine2'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['b2'] = self.layers['Affine2'].db
        return grads
    
    # ファイルへの保存
    def save_params(self, filename):
        np.savez(filename, W1=self.params['W1'], W2=self.params['W2'], b1=self.params['b1'], b2=self.params['b2'])
    
    # ファイルからの読み込み
    def load_params(self, filename):
        npz = np.load(filename)
        self.params['W1'] = npz['W1']
        self.params['W2'] = npz['W2']
        self.params['b1'] = npz['b1']
        self.params['b2'] = npz['b2']
        self.layers['Affine1'].W = self.params['W1']
        self.layers['Affine1'].b = self.params['b1']
        self.layers['Affine2'].W = self.params['W2']
        self.layers['Affine2'].b = self.params['b2']
    
    # テストデータを用いた精度確認
    def test_acc(self, x, t):
        ok = 0
        ng = 0
        for i in range(x.shape[0]):
            xx = x[[i]].reshape((1, d))
            tt = t[[i]]
            y, e = self.forward(xx, tt, False)
            res = np.argmax(y)
            ans = np.argmax(tt)
            if res == ans:
                ok += 1
            else:
                ng += 1
        print("正答数 : {}".format(ok))
        print("誤答数 : {}".format(ng))
        print("正答率 : {}%".format(float(ok) * 100.0 / float(ok + ng)))

## Sample NN 2

In [10]:
class AdvancedNN:
    def __init__(self, d, m, c):
        # パラメータ初期化
        self.params = {}
        self.params['W1'] = he(d, (d, m))
        self.params['W2'] = he(m, (m, c))
        self.params['Gamma'] = 1.0
        self.params['Beta'] = 0.0
        self.params['b1'] = he(d, (1, m))
        self.params['b2'] = he(m, (1, c))
        # NN生成
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['BatchNormalization'] = BatchNormalization(self.params['Gamma'], self.params['Beta'])
        self.layers['ReLU'] = ReLU()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithCrossEntropy()
        
    # 順伝播
    def forward(self, x, t, tflag=True):
        for layer in self.layers.values():
            x = layer.forward(x, train_flag=tflag)
        y, loss = self.lastLayer.forward(x, t)
        return y, loss
    
    # 逆伝播
    def backpropagation(self):
        dy = 1.0
        dy = self.lastLayer.backward(dy)
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dy = layer.backward(dy)
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['W2'] = self.layers['Affine2'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['b2'] = self.layers['Affine2'].db
        grads['Gamma'] = self.layers['BatchNormalization'].dgamma
        grads['Beta'] = self.layers['BatchNormalization'].dbeta
        return grads
    
    # ファイルへの保存
    def save_params(self, filename):
        np.savez(filename, W1=self.params['W1'], W2=self.params['W2'], 
                 b1=self.params['b1'], b2=self.params['b2'], 
                 Gamma=self.params['Gamma'], Beta=self.params['Beta'],
                 BN_T_M=self.layers['BatchNormalization'].test_mean,
                 BN_T_V=self.layers['BatchNormalization'].test_var)
    
    # ファイルからの読み込み
    def load_params(self, filename):
        npz = np.load(filename)
        self.params['W1'] = npz['W1']
        self.params['W2'] = npz['W2']
        self.params['b1'] = npz['b1']
        self.params['b2'] = npz['b2']
        self.params['Gamma'] = npz['Gamma']
        self.params['Beta'] = npz['Beta']
        bn_test_mean = npz['BN_T_M']
        bn_test_var = npz['BN_T_V']
        self.layers['Affine1'].W = self.params['W1']
        self.layers['Affine1'].b = self.params['b1']
        self.layers['Affine2'].W = self.params['W2']
        self.layers['Affine2'].b = self.params['b2']
        self.layers['BatchNormalization'].gamma = self.params['Gamma']
        self.layers['BatchNormalization'].beta = self.params['Beta']
        self.layers['BatchNormalization'].test_mean = bn_test_mean
        self.layers['BatchNormalization'].test_var = bn_test_var
    
    # テストデータを用いた精度確認
    def test_acc(self, x, t):
        ok = 0
        ng = 0
        for i in range(x.shape[0]):
            xx = x[[i]].reshape((1, d))
            tt = t[[i]]
            y, e = self.forward(xx, tt, False)
            res = np.argmax(y)
            ans = np.argmax(tt)
            if res == ans:
                ok += 1
            else:
                ng += 1
        print("正答数 : {}".format(ok))
        print("誤答数 : {}".format(ng))
        print("正答率 : {}%".format(float(ok) * 100.0 / float(ok + ng)))

## Sample CNN

In [11]:
class AdvancedCNN:
    def __init__(self, input_dim=(1,28,28), hidden_size=50, output_size=10,
                 cparam={'filter_size':30, 'filter_h':5, 'filter_w':5, 'stride':1, 'padding':0}):
        conv_output_size = (input_dim[1] - cparam['filter_h'] + 2 * cparam['padding']) / cparam['stride'] + 1
        pool_output_size = int(cparam['filter_size'] * (conv_output_size / 2) * (conv_output_size / 2))
        # パラメータ初期化
        self.params = {}
        self.params['W1'] = he(input_dim[0] * input_dim[1] * input_dim[2], (cparam['filter_size'], input_dim[0], cparam['filter_h'], cparam['filter_w']))
        self.params['W2'] = he(pool_output_size, (pool_output_size, hidden_size))
        self.params['W3'] = he(hidden_size, (hidden_size, output_size))
        self.params['b1'] = np.zeros(cparam['filter_size'])
        self.params['b2'] = np.zeros(hidden_size)
        self.params['b3'] = np.zeros(output_size)
        # NN生成
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], cparam['stride'], cparam['padding'])
        self.layers['ReLU'] = ReLU()
        self.layers['Pool1'] = Pooling(2, 2, 2, 0)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['ReLU'] = ReLU()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        self.lastLayer = SoftmaxWithCrossEntropy()
        
    # 順伝播
    def forward(self, x, t, tflag=True):
        for layer in self.layers.values():
            x = layer.forward(x, train_flag=tflag)
        y, loss = self.lastLayer.forward(x, t)
        return y, loss
    
    # 逆伝播
    def backpropagation(self):
        dy = 1.0
        dy = self.lastLayer.backward(dy)
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dy = layer.backward(dy)
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['W2'] = self.layers['Affine1'].dW
        grads['W3'] = self.layers['Affine2'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['b2'] = self.layers['Affine1'].db
        grads['b3'] = self.layers['Affine2'].db
        return grads
    
    # ファイルへの保存
    def save_params(self, filename):
        np.savez(filename, W1=self.params['W1'], W2=self.params['W2'], W3=self.params['W3'], 
                 b1=self.params['b1'], b2=self.params['b2'], b3=self.params['b3'])
    
    # ファイルからの読み込み
    def load_params(self, filename):
        npz = np.load(filename)
        self.params['W1'] = npz['W1']
        self.params['W2'] = npz['W2']
        self.params['W3'] = npz['W3']
        self.params['b1'] = npz['b1']
        self.params['b2'] = npz['b2']
        self.params['b3'] = npz['b3']
        self.layers['Conv1'].W = self.params['W1']
        self.layers['Conv1'].b = self.params['b1']
        self.layers['Affine1'].W = self.params['W2']
        self.layers['Affine1'].b = self.params['b2']
        self.layers['Affine2'].W = self.params['W3']
        self.layers['Affine2'].b = self.params['b3']
    
    # テストデータを用いた精度確認
    def test_acc(self, x, t):
        ok = 0
        ng = 0
        for i in range(x.shape[0]):
            xx = x[[i]]
            tt = t[[i]]
            y, e = self.forward(xx, tt, False)
            res = np.argmax(y)
            ans = np.argmax(tt)
            if res == ans:
                ok += 1
            else:
                ng += 1
        print("正答数 : {}".format(ok))
        print("誤答数 : {}".format(ng))
        print("正答率 : {}%".format(float(ok) * 100.0 / float(ok + ng)))

## Minibatch

In [12]:
# ミニバッチ学習
def minibatch(network, optimizer, filename, is_cnn=False):
    for i in range(epoch):
        e = 0.0
        for _ in range(int(n / b)):
            # ランダムに画像を選ぶ
            index = np.random.choice(n, b, replace=True)
            x = X_train[index]
            t = T_train[index]
            # CNNでないなら入力の形を整える
            if not is_cnn:
                x = x.reshape((b, d))
                # t = t.reshape((b, c))
            # 順伝播
            y, e = network.forward(x, t)
            # 誤差逆伝播法
            grads = network.backpropagation()
            # パラメータ更新
            optimizer.update(network.params, grads)
        print("{}エポック目の誤差平均 : {}".format(i + 1, e / (n / b)))
    network.save_params(filename)

## Sample Learning

In [13]:
# 3層NNによる学習の場合（SGD）
minibatch(NormalNN(d, m, c), SGD(), 'data/nn_sgd')

# 精度確認
nn = NormalNN(d, m, c)
nn.load_params('data/nn_sgd.npz')
nn.test_acc(X_test, T_test)

1エポック目の誤差平均 : 0.003230982294974653
2エポック目の誤差平均 : 0.002430484251462343
3エポック目の誤差平均 : 0.001944492544372093
4エポック目の誤差平均 : 0.0015860593952860592
5エポック目の誤差平均 : 0.0013823642708127495
6エポック目の誤差平均 : 0.0012487544986569145
7エポック目の誤差平均 : 0.0009918239441929243
8エポック目の誤差平均 : 0.0010450638999146267
9エポック目の誤差平均 : 0.000997694291884218
10エポック目の誤差平均 : 0.0008123697800993858
11エポック目の誤差平均 : 0.0007987189547510484
12エポック目の誤差平均 : 0.0007259514270028853
13エポック目の誤差平均 : 0.0009560844296470657
14エポック目の誤差平均 : 0.0006584579416929816
15エポック目の誤差平均 : 0.0006301444338584097
16エポック目の誤差平均 : 0.0006471637735964611
17エポック目の誤差平均 : 0.00040680491304802177
18エポック目の誤差平均 : 0.00046733768513873065
19エポック目の誤差平均 : 0.0005460591719597601
20エポック目の誤差平均 : 0.0005025941471894595
21エポック目の誤差平均 : 0.000596715013404952
22エポック目の誤差平均 : 0.000619088105322979
23エポック目の誤差平均 : 0.0005721612999925449
24エポック目の誤差平均 : 0.000516929867583436
25エポック目の誤差平均 : 0.0005084251954249819
26エポック目の誤差平均 : 0.00042112745305897714
27エポック目の誤差平均 : 0.0004270590858410093
28エポック目の誤差平均 :

In [14]:
# 3層NNによる学習の場合（MomentumSGD）
minibatch(NormalNN(d, m, c), MomentumSGD(), 'data/nn_momentumsgd')

# 精度確認
nn = NormalNN(d, m, c)
nn.load_params('data/nn_momentumsgd.npz')
nn.test_acc(X_test, T_test)

1エポック目の誤差平均 : 0.0009170589151046058
2エポック目の誤差平均 : 0.0004922766698682133
3エポック目の誤差平均 : 0.0005069294970901675
4エポック目の誤差平均 : 0.0004987108929336217
5エポック目の誤差平均 : 0.0003005100915198889
6エポック目の誤差平均 : 0.0005848081518004932
7エポック目の誤差平均 : 0.00029547253762649684
8エポック目の誤差平均 : 0.0005371116658460112
9エポック目の誤差平均 : 0.0003707289058919259
10エポック目の誤差平均 : 0.00045416597064473134
11エポック目の誤差平均 : 0.00033057109533457047
12エポック目の誤差平均 : 0.0005201103881766894
13エポック目の誤差平均 : 0.0003884829560452636
14エポック目の誤差平均 : 0.0003303720501580523
15エポック目の誤差平均 : 0.0003320998903524091
16エポック目の誤差平均 : 0.0002654708110043512
17エポック目の誤差平均 : 0.0002960487663365769
18エポック目の誤差平均 : 0.00016340513997812058
19エポック目の誤差平均 : 0.00011523825688937343
20エポック目の誤差平均 : 0.0002106727818665434
21エポック目の誤差平均 : 0.00015044828064279863
22エポック目の誤差平均 : 0.0003232454846308064
23エポック目の誤差平均 : 0.0003443700165041184
24エポック目の誤差平均 : 0.00021194715288531238
25エポック目の誤差平均 : 0.0002755369661250295
26エポック目の誤差平均 : 0.0002794599850470879
27エポック目の誤差平均 : 0.0003239873964497166
28エ

In [15]:
# 3層NNによる学習の場合（AdaGrad）
minibatch(NormalNN(d, m, c), AdaGrad(), 'data/nn_adagrad')

# 精度確認
nn = NormalNN(d, m, c)
nn.load_params('data/nn_adagrad.npz')
nn.test_acc(X_test, T_test)

1エポック目の誤差平均 : 0.002699746037917266
2エポック目の誤差平均 : 0.0024076876628265505
3エポック目の誤差平均 : 0.0020437765571028306
4エポック目の誤差平均 : 0.0019217827332446355
5エポック目の誤差平均 : 0.0018491958924067105
6エポック目の誤差平均 : 0.001973900814797447
7エポック目の誤差平均 : 0.0017190891306828248
8エポック目の誤差平均 : 0.001485221980465757
9エポック目の誤差平均 : 0.0015139644443923933
10エポック目の誤差平均 : 0.0013623652596518943
11エポック目の誤差平均 : 0.001470994912059193
12エポック目の誤差平均 : 0.0012590992787248974
13エポック目の誤差平均 : 0.0013434231871932033
14エポック目の誤差平均 : 0.0012872251924871847
15エポック目の誤差平均 : 0.0013145679301937597
16エポック目の誤差平均 : 0.0011605573571003478
17エポック目の誤差平均 : 0.0011123032784942448
18エポック目の誤差平均 : 0.0011482546821854202
19エポック目の誤差平均 : 0.0012028306010702541
20エポック目の誤差平均 : 0.0010751451753975704
21エポック目の誤差平均 : 0.0012194055376808367
22エポック目の誤差平均 : 0.0011355469987826795
23エポック目の誤差平均 : 0.0009393366866362597
24エポック目の誤差平均 : 0.0011071817940525913
25エポック目の誤差平均 : 0.001162630875411259
26エポック目の誤差平均 : 0.0011392433743602631
27エポック目の誤差平均 : 0.0010071370067997916
28エポック目の誤差平均 : 

In [16]:
# 3層NNによる学習の場合（RMSProp）
minibatch(NormalNN(d, m, c), RMSProp(), 'data/nn_rmsprop')

# 精度確認
nn = NormalNN(d, m, c)
nn.load_params('data/nn_rmsprop.npz')
nn.test_acc(X_test, T_test)

1エポック目の誤差平均 : 0.0006107484850971527
2エポック目の誤差平均 : 0.00036151545668083196
3エポック目の誤差平均 : 0.0004577269076388583
4エポック目の誤差平均 : 0.00022868240412386747
5エポック目の誤差平均 : 0.0002901181780800104
6エポック目の誤差平均 : 0.00027808058405544305
7エポック目の誤差平均 : 0.00021577197356564186
8エポック目の誤差平均 : 0.00018086362296312774
9エポック目の誤差平均 : 0.00014608684264499734
10エポック目の誤差平均 : 0.00012957332354950364
11エポック目の誤差平均 : 7.05397362230752e-05
12エポック目の誤差平均 : 0.00023551850724913375
13エポック目の誤差平均 : 0.0001144605460927488
14エポック目の誤差平均 : 9.877202730979158e-05
15エポック目の誤差平均 : 6.387800674453211e-05
16エポック目の誤差平均 : 3.560283544964944e-05
17エポック目の誤差平均 : 7.180016719589282e-05
18エポック目の誤差平均 : 9.458409588069973e-05
19エポック目の誤差平均 : 0.000116684268094468
20エポック目の誤差平均 : 3.6503578344969984e-05
21エポック目の誤差平均 : 0.0001123139804153818
22エポック目の誤差平均 : 4.446599293003225e-05
23エポック目の誤差平均 : 0.0001035514743994188
24エポック目の誤差平均 : 4.674150134740644e-05
25エポック目の誤差平均 : 4.543069337253908e-05
26エポック目の誤差平均 : 7.34526214778222e-05
27エポック目の誤差平均 : 4.355560840658794e-05
28エポ

In [17]:
# 3層NNによる学習の場合（AdaDelta）
minibatch(NormalNN(d, m, c), AdaDelta(), 'data/nn_adadelta')

# 精度確認
nn = NormalNN(d, m, c)
nn.load_params('data/nn_adadelta.npz')
nn.test_acc(X_test, T_test)

1エポック目の誤差平均 : 0.0003696555696290338
2エポック目の誤差平均 : 0.0002674554819990995
3エポック目の誤差平均 : 0.00044525392508610977
4エポック目の誤差平均 : 8.871750456502476e-05
5エポック目の誤差平均 : 0.00016565361945473936
6エポック目の誤差平均 : 0.00036773114363902216
7エポック目の誤差平均 : 0.00022616711815137274
8エポック目の誤差平均 : 0.0002840510843193345
9エポック目の誤差平均 : 0.0001294997390013683
10エポック目の誤差平均 : 4.333845725743318e-05
11エポック目の誤差平均 : 8.284190287762127e-05
12エポック目の誤差平均 : 5.6324458865087554e-05
13エポック目の誤差平均 : 7.242155894988428e-05
14エポック目の誤差平均 : 0.00010365674217056174
15エポック目の誤差平均 : 0.00024118980228985458
16エポック目の誤差平均 : 8.452890040396287e-05
17エポック目の誤差平均 : 7.446230695829858e-05
18エポック目の誤差平均 : 8.439469706186386e-05
19エポック目の誤差平均 : 4.932371849083162e-05
20エポック目の誤差平均 : 0.00013916332196908985
21エポック目の誤差平均 : 3.7198701304758195e-05
22エポック目の誤差平均 : 6.73820220184961e-05
23エポック目の誤差平均 : 0.00011926777077621603
24エポック目の誤差平均 : 5.201130422788114e-05
25エポック目の誤差平均 : 5.1158630416826354e-05
26エポック目の誤差平均 : 4.330485882611456e-05
27エポック目の誤差平均 : 2.4029761961964918e-05

In [18]:
# 3層NNによる学習の場合（Adam）
minibatch(NormalNN(d, m, c), Adam(), 'data/nn_adam')

# 精度確認
nn = NormalNN(d, m, c)
nn.load_params('data/nn_adam.npz')
nn.test_acc(X_test, T_test)

1エポック目の誤差平均 : 0.0007296712235649719
2エポック目の誤差平均 : 0.0002571260343618341
3エポック目の誤差平均 : 0.00041331072209553556
4エポック目の誤差平均 : 0.00037656193624252055
5エポック目の誤差平均 : 0.00042374682931320304
6エポック目の誤差平均 : 0.00020112748163642524
7エポック目の誤差平均 : 0.0002534457770944605
8エポック目の誤差平均 : 0.0002627248285893633
9エポック目の誤差平均 : 0.00012990757665279194
10エポック目の誤差平均 : 0.00018507210281015939
11エポック目の誤差平均 : 7.67619165076512e-05
12エポック目の誤差平均 : 8.71444621943942e-05
13エポック目の誤差平均 : 0.00035254946329446704
14エポック目の誤差平均 : 0.00011337848661489618
15エポック目の誤差平均 : 6.00244508718523e-05
16エポック目の誤差平均 : 5.988085234177581e-05
17エポック目の誤差平均 : 9.15693720341259e-05
18エポック目の誤差平均 : 0.00017976875142821432
19エポック目の誤差平均 : 5.051794565212939e-05
20エポック目の誤差平均 : 5.9931559706469966e-05
21エポック目の誤差平均 : 8.532454319590736e-05
22エポック目の誤差平均 : 0.0001235266477946897
23エポック目の誤差平均 : 8.145522999191287e-05
24エポック目の誤差平均 : 2.7634597137849618e-05
25エポック目の誤差平均 : 2.511013902557498e-05
26エポック目の誤差平均 : 3.2276286336446836e-05
27エポック目の誤差平均 : 8.868079927479484e-05
28

In [19]:
# 3層NNによる学習の場合（ReLU + BatchNormalization + AdaDelta）
minibatch(AdvancedNN(d, m, c), AdaDelta(), 'data/ann_bn_adadelta')

# 精度確認
nn = AdvancedNN(d, m, c)
nn.load_params('data/ann_bn_adadelta.npz')
nn.test_acc(X_test, T_test)

1エポック目の誤差平均 : 0.0008005042421458591
2エポック目の誤差平均 : 0.0006284837875793833
3エポック目の誤差平均 : 0.0003554889432052642
4エポック目の誤差平均 : 0.00021164253999073174
5エポック目の誤差平均 : 0.00015563673224064584
6エポック目の誤差平均 : 0.00029648501331076805
7エポック目の誤差平均 : 0.0001925971492468666
8エポック目の誤差平均 : 0.00026221754639713644
9エポック目の誤差平均 : 0.00032117175876731583
10エポック目の誤差平均 : 7.651652302728934e-05
11エポック目の誤差平均 : 0.00010888380823125426
12エポック目の誤差平均 : 7.768248568942125e-05
13エポック目の誤差平均 : 0.00020810921180754906
14エポック目の誤差平均 : 7.826707649080612e-05
15エポック目の誤差平均 : 0.00016151644129294365
16エポック目の誤差平均 : 6.60239512785564e-05
17エポック目の誤差平均 : 6.656579415848662e-05
18エポック目の誤差平均 : 5.785200967487481e-05
19エポック目の誤差平均 : 0.00010915171241448725
20エポック目の誤差平均 : 6.373462541877408e-05
21エポック目の誤差平均 : 7.058187067439193e-05
22エポック目の誤差平均 : 8.11965595003555e-05
23エポック目の誤差平均 : 0.00014538530797395148
24エポック目の誤差平均 : 0.00020123535885998423
25エポック目の誤差平均 : 7.80914719389644e-05
26エポック目の誤差平均 : 7.278606909204043e-05
27エポック目の誤差平均 : 5.837302546900465e-05
28

In [20]:
# CNNによる学習の場合
# かなり時間がかかる
# minibatch(AdvancedCNN(), RMSProp(), 'data/advanced_cnn', True)

# 精度確認
cnn = AdvancedCNN()
cnn.load_params('data/advanced_cnn.npz')
cnn.test_acc(X_test, T_test)

正答数 : 9865
誤答数 : 135
正答率 : 98.65%
